In [111]:
import pandas as pd
import numpy as np

## Clean and transform time series data

In [112]:
# we will only perform the clustering on the close prices, drop all other columns
df = pd.read_csv('../../data/sp500_all_companies.csv', header=[0, 1], index_col=0)

# clean the data and get the close and open prices
df = df.loc[:, ((df.columns.get_level_values(0) != 'High') | (df.columns.get_level_values(0) != 'Low'))] # extract the open price also
df.columns = [f'{col[0]}_{col[1]}' for col in df.columns]

df = df.reset_index()
# Melt the DataFrame to long format
df_long = df.melt(id_vars='Date', var_name='feature_ticker', value_name='value')

# Split 'feature_ticker' into 'feature' and 'Ticker'
df_long[['feature', 'ticker']] = df_long['feature_ticker'].str.split('_', n=1, expand=True)

df_long['value'] = (
    df_long.sort_values(['ticker', 'feature', 'Date'])
          .groupby(['ticker', 'feature'])['value']
          .transform(lambda x: x.interpolate(method='quadratic', limit_direction='both'))
)

In [113]:
df_panel = df_long.pivot(index=['ticker', 'Date'], columns='feature', values='value')
df_panel

feature                 Close  Dividends        High         Low        Open  \
ticker Date                                                                    
A      2015-05-04   38.636948        0.0   38.765925   38.415851   38.581675   
       2015-05-05   38.314510        0.0   38.554032   38.194748   38.434273   
       2015-05-06   38.314510        0.0   38.563246   38.102625   38.563246   
       2015-05-07   38.507973        0.0   38.554034   38.056565   38.314512   
       2015-05-08   39.152843        0.0   39.180480   38.885682   38.904107   
...                       ...        ...         ...         ...         ...   
ZTS    2025-04-28  153.570007        0.0  154.779999  152.449997  153.350006   
       2025-04-29  155.429993        0.0  156.220001  152.250000  153.289993   
       2025-04-30  156.399994        0.0  157.199997  153.770004  155.720001   
       2025-05-01  155.649994        0.0  157.419998  152.300003  156.550003   
       2025-05-02  157.440002        0.0  159.970001  156.679993  158.490005   

feature            Stock Splits     Volume  
ticker Date                                 
A      2015-05-04           0.0  1073800.0  
       2015-05-05           0.0  1254800.0  
       2015-05-06           0.0   934600.0  
       2015-05-07           0.0   999700.0  
       2015-05-08           0.0  1033800.0  
...                         ...        ...  
ZTS    2025-04-28           0.0  1873400.0  
       2025-04-29           0.0  2065100.0  
       2025-04-30           0.0  2926200.0  
       2025-05-01           0.0  1930300.0  
       2025-05-02           0.0  2654700.0  

[1265548 rows x 7 columns]

In [114]:
na_frac = df_panel.isna().groupby('ticker').mean().max(axis=1)

# Keep only tickers with <= 20% NA values
tickers_to_keep = na_frac[na_frac <= 0.2].index

df_panel_filtered = df_panel.loc[tickers_to_keep]

df_panel_interpolated = (
    df_panel_filtered
    .groupby('ticker')
    .apply(lambda group: group.interpolate(method='linear', limit_direction='both'))
    .droplevel(0)
)

df_panel_interpolated

feature                 Close  Dividends        High         Low        Open  \
ticker Date                                                                    
A      2015-05-04   38.636948        0.0   38.765925   38.415851   38.581675   
       2015-05-05   38.314510        0.0   38.554032   38.194748   38.434273   
       2015-05-06   38.314510        0.0   38.563246   38.102625   38.563246   
       2015-05-07   38.507973        0.0   38.554034   38.056565   38.314512   
       2015-05-08   39.152843        0.0   39.180480   38.885682   38.904107   
...                       ...        ...         ...         ...         ...   
ZTS    2025-04-28  153.570007        0.0  154.779999  152.449997  153.350006   
       2025-04-29  155.429993        0.0  156.220001  152.250000  153.289993   
       2025-04-30  156.399994        0.0  157.199997  153.770004  155.720001   
       2025-05-01  155.649994        0.0  157.419998  152.300003  156.550003   
       2025-05-02  157.440002        0.0  159.970001  156.679993  158.490005   

feature            Stock Splits     Volume  
ticker Date                                 
A      2015-05-04           0.0  1073800.0  
       2015-05-05           0.0  1254800.0  
       2015-05-06           0.0   934600.0  
       2015-05-07           0.0   999700.0  
       2015-05-08           0.0  1033800.0  
...                         ...        ...  
ZTS    2025-04-28           0.0  1873400.0  
       2025-04-29           0.0  2065100.0  
       2025-04-30           0.0  2926200.0  
       2025-05-01           0.0  1930300.0  
       2025-05-02           0.0  2654700.0  

[1207680 rows x 7 columns]

In [115]:
df_panel_interpolated.columns.name = None

In [116]:
df_panel_interpolated.columns = [i.lower() for i in df_panel_interpolated.columns]

In [117]:
df_panel_interpolated

close  dividends        high         low        open  \
ticker Date                                                                    
A      2015-05-04   38.636948        0.0   38.765925   38.415851   38.581675   
       2015-05-05   38.314510        0.0   38.554032   38.194748   38.434273   
       2015-05-06   38.314510        0.0   38.563246   38.102625   38.563246   
       2015-05-07   38.507973        0.0   38.554034   38.056565   38.314512   
       2015-05-08   39.152843        0.0   39.180480   38.885682   38.904107   
...                       ...        ...         ...         ...         ...   
ZTS    2025-04-28  153.570007        0.0  154.779999  152.449997  153.350006   
       2025-04-29  155.429993        0.0  156.220001  152.250000  153.289993   
       2025-04-30  156.399994        0.0  157.199997  153.770004  155.720001   
       2025-05-01  155.649994        0.0  157.419998  152.300003  156.550003   
       2025-05-02  157.440002        0.0  159.970001  156.679993  158.490005   

                   stock splits     volume  
ticker Date                                 
A      2015-05-04           0.0  1073800.0  
       2015-05-05           0.0  1254800.0  
       2015-05-06           0.0   934600.0  
       2015-05-07           0.0   999700.0  
       2015-05-08           0.0  1033800.0  
...                         ...        ...  
ZTS    2025-04-28           0.0  1873400.0  
       2025-04-29           0.0  2065100.0  
       2025-04-30           0.0  2926200.0  
       2025-05-01           0.0  1930300.0  
       2025-05-02           0.0  2654700.0  

[1207680 rows x 7 columns]

In [118]:
df_panel_interpolated = df_panel_interpolated.rename_axis(['ticker', 'date'])

In [119]:
df_panel_interpolated

close  dividends        high         low        open  \
ticker date                                                                    
A      2015-05-04   38.636948        0.0   38.765925   38.415851   38.581675   
       2015-05-05   38.314510        0.0   38.554032   38.194748   38.434273   
       2015-05-06   38.314510        0.0   38.563246   38.102625   38.563246   
       2015-05-07   38.507973        0.0   38.554034   38.056565   38.314512   
       2015-05-08   39.152843        0.0   39.180480   38.885682   38.904107   
...                       ...        ...         ...         ...         ...   
ZTS    2025-04-28  153.570007        0.0  154.779999  152.449997  153.350006   
       2025-04-29  155.429993        0.0  156.220001  152.250000  153.289993   
       2025-04-30  156.399994        0.0  157.199997  153.770004  155.720001   
       2025-05-01  155.649994        0.0  157.419998  152.300003  156.550003   
       2025-05-02  157.440002        0.0  159.970001  156.679993  158.490005   

                   stock splits     volume  
ticker date                                 
A      2015-05-04           0.0  1073800.0  
       2015-05-05           0.0  1254800.0  
       2015-05-06           0.0   934600.0  
       2015-05-07           0.0   999700.0  
       2015-05-08           0.0  1033800.0  
...                         ...        ...  
ZTS    2025-04-28           0.0  1873400.0  
       2025-04-29           0.0  2065100.0  
       2025-04-30           0.0  2926200.0  
       2025-05-01           0.0  1930300.0  
       2025-05-02           0.0  2654700.0  

[1207680 rows x 7 columns]

In [120]:
df_panel_interpolated.to_csv('../../data/sp500_all_companies_cleaned.csv')

## Read in sector information for potential clustering

In [121]:
sector = pd.read_csv('../../data/sp500_companies_sector.csv', index_col=0)

In [122]:
sector

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [123]:
sector = sector[['Symbol', 'GICS Sector']]
sector['Symbol'] = sector['Symbol'].str.replace('.', '-')

C:\Users\Home\AppData\Local\Temp\ipykernel_10900\3195797137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sector['Symbol'] = sector['Symbol'].str.replace('.', '-')


In [124]:
sector['GICS Sector'].value_counts()

GICS Sector
Industrials               78
Financials                73
Information Technology    69
Health Care               60
Consumer Discretionary    51
Consumer Staples          38
Utilities                 31
Real Estate               31
Materials                 26
Communication Services    23
Energy                    23
Name: count, dtype: int64

In [125]:
df_panel_interpolated = df_panel_interpolated.reset_index()

In [128]:
df_panel_interpolated = df_panel_interpolated.set_index(['ticker', 'date'])

## Final checks

In [129]:
result = []
for ticker in df_panel_interpolated.index.get_level_values('ticker').unique():
    df_ticker = df_panel_interpolated.xs(ticker, level='ticker')
    df_ticker.index = pd.DatetimeIndex(df_ticker.index)
    df_ticker = df_ticker.asfreq('B')
    df_ticker = df_ticker.interpolate(method='linear', limit_direction='both')
    df_ticker['ticker'] = ticker
    result.append(df_ticker)

merged_bfreq = pd.concat(result)
merged_bfreq = merged_bfreq.set_index('ticker', append=True).reorder_levels(['ticker', 'date']).sort_index()

In [130]:
merged_bfreq = merged_bfreq.reset_index()
merged_bfreq  = pd.merge(
    merged_bfreq,
    sector.rename(columns={'Symbol': 'ticker'}),
    on='ticker',
    how='left'
)

In [131]:
merged_bfreq.isna().sum()

ticker          0
date            0
close           0
dividends       0
high            0
low             0
open            0
stock splits    0
volume          0
GICS Sector     0
dtype: int64

In [134]:
merged_bfreq.set_index(['ticker', 'date'], inplace=True)

In [135]:
merged_bfreq.to_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')